In [ ]:
#pip install pandas numpy matplotlib

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt


In [ ]:
# Step 1
df=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [5]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [6]:
duplicated=df.duplicated()


In [7]:
duplicated.count()

np.int64(7043)

In [8]:
df['Contract'].unique()

<StringArray>
['Month-to-month', 'One year', 'Two year']
Length: 3, dtype: str

In [9]:
# STEP 2: Remove Duplicates
df = df.drop_duplicates(subset=['customerID'], keep='first')

# STEP 3: Handle TotalCharges
# Convert to numeric, coerce errors to NaN
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

# Fill missing TotalCharges with calculated value
df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'])

# STEP 4: Standardize Contract
contract_mapping = {
    'Month-to-month': 'Month-to-Month',
    'Monthly': 'Month-to-Month',
    'One year': 'One Year',
    '1 year': 'One Year',
    'Two year': 'Two Year',
    '2 years': 'Two Year'
}
df['Contract'] = df['Contract'].str.strip().replace(contract_mapping)


In [10]:
# STEP 5: Standardize Payment Method
payment_mapping = {
    'Electronic check': 'Electronic Check',
    'electronic check': 'Electronic Check',
    'Mailed check': 'Mailed Check',
    'mailed check': 'Mailed Check',
    'Bank transfer (automatic)': 'Bank Transfer (Automatic)',
    'Credit card (automatic)': 'Credit Card (Automatic)'
}
df['PaymentMethod'] = df['PaymentMethod'].str.strip().replace(payment_mapping)

# STEP 6: Standardize Gender
df['gender'] = df['gender'].str.strip().str.capitalize()

# STEP 7: Standardize Yes/No fields
yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in yes_no_columns:
    df[col] = df[col].str.strip().str.capitalize()

# STEP 8: Handle service-related columns
service_columns = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                   'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for col in service_columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace({
        'No internet service': 'No',
        'No phone service': 'No',
        'yes': 'Yes',
        'no': 'No'
    })

In [11]:
# STEP 9: Standardize Internet Service
df['InternetService'] = df['InternetService'].replace({
    'DSL': 'DSL',
    'Fiber optic': 'Fiber Optic',
    'No': 'No'
})

# STEP 10: Add Calculated Columns
df['ChurnFlag'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
df['SeniorCitizenLabel'] = df['SeniorCitizen'].apply(lambda x: 'Yes' if x == 1 else 'No')

# Tenure Groups
def categorize_tenure(tenure):
    if tenure <= 12:
        return '0-1 Year'
    elif tenure <= 24:
        return '1-2 Years'
    elif tenure <= 36:
        return '2-3 Years'
    elif tenure <= 48:
        return '3-4 Years'
    elif tenure <= 60:
        return '4-5 Years'
    else:
        return '5+ Years'

df['TenureGroup'] = df['tenure'].apply(categorize_tenure)

# Monthly Charge Categories
def categorize_charges(charge):
    if charge < 30:
        return 'Low (< $30)'
    elif charge < 70:
        return 'Medium ($30-$70)'
    else:
        return 'High (> $70)'

df['MonthlyChargeCategory'] = df['MonthlyCharges'].apply(categorize_charges)

# Revenue Lost
df['RevenueLost'] = df.apply(lambda row: row['TotalCharges'] if row['ChurnFlag'] == 1 else 0, axis=1)

# Service Flags
df['HasInternetService'] = df['InternetService'].apply(lambda x: 'No' if x == 'No' else 'Yes')
df['HasPhoneService'] = df['PhoneService'].apply(lambda x: 'No' if x == 'No' else 'Yes')

# STEP 11: Data Validation
print("Data Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
print("\nContract Values:", df['Contract'].unique())
print("\nPayment Method Values:", df['PaymentMethod'].unique())
print("\nChurn Values:", df['Churn'].unique())
print("\nData Types:\n", df.dtypes)

# STEP 12: Export Cleaned Data
df.to_csv('Telco_Customer_Churn_CLEANED.csv', index=False)
print("\n✓ Cleaned data exported successfully!")

Data Shape: (7043, 28)

Missing Values:
 customerID               0
gender                   0
SeniorCitizen            0
Partner                  0
Dependents               0
tenure                   0
PhoneService             0
MultipleLines            0
InternetService          0
OnlineSecurity           0
OnlineBackup             0
DeviceProtection         0
TechSupport              0
StreamingTV              0
StreamingMovies          0
Contract                 0
PaperlessBilling         0
PaymentMethod            0
MonthlyCharges           0
TotalCharges             0
Churn                    0
ChurnFlag                0
SeniorCitizenLabel       0
TenureGroup              0
MonthlyChargeCategory    0
RevenueLost              0
HasInternetService       0
HasPhoneService          0
dtype: int64

Contract Values: <StringArray>
['Month-to-Month', 'One Year', 'Two Year']
Length: 3, dtype: str

Payment Method Values: <StringArray>
[         'Electronic Check',              'Mailed Chec

import pandas as pd
import numpy as np

# STEP 1: Load Data
# What: Import the raw Telco customer churn CSV file into a DataFrame
# Why: Establish foundation for analysis and enable programmatic data manipulation
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# STEP 2: Remove Duplicates
# What: Eliminate duplicate customerID records, keeping only the first occurrence
# Why: Ensure data integrity so each customer appears once, preventing inflated counts and skewed churn rates
df = df.drop_duplicates(subset=['customerID'], keep='first')

# STEP 3: Handle TotalCharges
# What: Convert TotalCharges from text to numeric, coercing errors to NaN, then fill missing values with calculated revenue
# Why: Enable accurate revenue calculations since text values break aggregations and blanks represent new customers
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'] * df['tenure'])

# STEP 4: Standardize Contract
# What: Unify contract type variations into 3 standard categories (Month-to-Month, One Year, Two Year)
# Why: Eliminate duplicate categories caused by inconsistent naming so charts show clean segments and filters work correctly
contract_mapping = {
    'Month-to-month': 'Month-to-Month',
    'Monthly': 'Month-to-Month',
    'One year': 'One Year',
    '1 year': 'One Year',
    'Two year': 'Two Year',
    '2 years': 'Two Year'
}
df['Contract'] = df['Contract'].str.strip().replace(contract_mapping)

# STEP 5: Standardize Payment Method
# What: Standardize payment method capitalization and remove text variations to create 4 clean categories
# Why: Prevent case-sensitive duplicates in visuals and enable accurate analysis of payment method impact on churn
payment_mapping = {
    'Electronic check': 'Electronic Check',
    'electronic check': 'Electronic Check',
    'Mailed check': 'Mailed Check',
    'mailed check': 'Mailed Check',
    'Bank transfer (automatic)': 'Bank Transfer (Automatic)',
    'Credit card (automatic)': 'Credit Card (Automatic)'
}
df['PaymentMethod'] = df['PaymentMethod'].str.strip().replace(payment_mapping)

# STEP 6: Standardize Gender
# What: Apply consistent capitalization to gender values (Male, Female)
# Why: Ensure professional presentation and uniform formatting across all categorical fields in dashboards
df['gender'] = df['gender'].str.strip().str.capitalize()

# STEP 7: Standardize Yes/No fields
# What: Standardize binary fields to exactly "Yes" or "No" by trimming spaces and capitalizing
# Why: Critical for accurate churn calculations and reliable DAX filtering since "yes" ≠ "Yes" in Power BI
yes_no_columns = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
for col in yes_no_columns:
    df[col] = df[col].str.strip().str.capitalize()

# STEP 8: Handle service-related columns
# What: Replace "No internet service" and "No phone service" with simple "No", standardize to Yes/No binary
# Why: Simplify analysis from 3 values to 2, making visualizations cleaner and service adoption rates easier to calculate
service_columns = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 
                   'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
for col in service_columns:
    df[col] = df[col].str.strip()
    df[col] = df[col].replace({
        'No internet service': 'No',
        'No phone service': 'No',
        'yes': 'Yes',
        'no': 'No'
    })

# STEP 9: Standardize Internet Service
# What: Ensure consistent capitalization for internet service types (DSL, Fiber Optic, No)
# Why: Enable accurate comparison of churn rates between DSL and Fiber Optic customers with professional formatting
df['InternetService'] = df['InternetService'].replace({
    'DSL': 'DSL',
    'Fiber optic': 'Fiber Optic',
    'No': 'No'
})

# STEP 10: Add Calculated Columns
# What: Create 7 new analytical columns including ChurnFlag, TenureGroup, ChargeCategory, and RevenueLost
# Why: Enable advanced segmentation, financial impact analysis, and provide ready-made fields for DAX measures and Key Influencers visual
df['ChurnFlag'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
df['SeniorCitizenLabel'] = df['SeniorCitizen'].apply(lambda x: 'Yes' if x == 1 else 'No')

def categorize_tenure(tenure):
    if tenure <= 12:
        return '0-1 Year'
    elif tenure <= 24:
        return '1-2 Years'
    elif tenure <= 36:
        return '2-3 Years'
    elif tenure <= 48:
        return '3-4 Years'
    elif tenure <= 60:
        return '4-5 Years'
    else:
        return '5+ Years'

df['TenureGroup'] = df['tenure'].apply(categorize_tenure)

def categorize_charges(charge):
    if charge < 30:
        return 'Low (< $30)'
    elif charge < 70:
        return 'Medium ($30-$70)'
    else:
        return 'High (> $70)'

df['MonthlyChargeCategory'] = df['MonthlyCharges'].apply(categorize_charges)

df['RevenueLost'] = df.apply(lambda row: row['TotalCharges'] if row['ChurnFlag'] == 1 else 0, axis=1)
df['HasInternetService'] = df['InternetService'].apply(lambda x: 'No' if x == 'No' else 'Yes')
df['HasPhoneService'] = df['PhoneService'].apply(lambda x: 'No' if x == 'No' else 'Yes')

# STEP 11: Data Validation
# What: Print summary statistics, unique values, and data types to verify cleaning success
# Why: Confirm all transformations worked correctly before exporting, catching any errors early
print("Data Shape:", df.shape)
print("\nMissing Values:\n", df.isnull().sum())
print("\nContract Values:", df['Contract'].unique())
print("\nPayment Method Values:", df['PaymentMethod'].unique())
print("\nChurn Values:", df['Churn'].unique())
print("\nData Types:\n", df.dtypes)

# STEP 12: Export Cleaned Data
# What: Save the fully cleaned and standardized dataset to a new CSV file
# Why: Preserve cleaned data for Power BI import, ensuring repeatable analysis and sharing with team members
df.to_csv('Telco_Customer_Churn_CLEANED.csv', index=False)
print("\n✓ Cleaned data exported successfully!")